<a href="https://colab.research.google.com/github/WendyGalvisL/repo-api-starter/blob/master/Prueba_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importar librerías

In [ ]:
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

from imblearn.over_sampling import SMOTE

# Cargar datos

In [ ]:
df = pd.read_csv('train_data.csv')
df.head()

,ID,Bankruptcy,Realized.Sales.Gross.Profit.Growth.Rate,Operating.Expense.Rate,Operating.Profit.Per.Share..Yuan...,Gross.Profit.to.Sales,Tax.rate..A.,Current.Liability.to.Liability,Continuous.Net.Profit.Growth.Rate,Long.term.Liability.to.Current.Assets,...,Per.Share.Net.profit.before.tax..Yuan...,Continuous.interest.rate..after.tax.,Liability.Assets.Flag,Working.Capital.to.Total.Assets,Degree.of.Financial.Leverage..DFL.,Operating.Gross.Margin,Contingent.liabilities.Net.worth,Operating.Profit.Growth.Rate,Cash.Flow.to.Liability,Quick.Assets.Total.Assets
0,1,0,0.087661,1.900349e-04,0.131094,0.583345,0.000000,0.815892,0.395719,0.089276,...,0.223720,0.801737,0,0.839112,0.036551,0.591350,0.007401,0.913175,0.489822,0.138714
1,2,0,0.021892,5.371576e-04,0.086167,0.695478,0.209649,0.768636,0.163505,0.000000,...,0.324763,0.791203,0,0.908450,0.126752,0.766066,0.020514,0.849241,0.550247,0.704211
2,3,0,0.102249,2.178329e-04,0.157755,0.766934,0.272061,0.643316,0.181571,0.010180,...,0.360892,0.803126,0,0.865943,0.018152,0.687211,0.004363,0.865814,0.447168,0.398086
3,4,0,0.029972,4.915756e+10,0.249279,0.611580,0.288357,0.752782,0.409196,0.018603,...,0.152445,0.824732,0,0.806254,0.051284,0.715973,0.014162,0.843357,0.485163,0.617386
4,6,0,0.021164,1.206001e-04,0.239817,0.614250,0.000000,0.559498,0.286989,0.148687,...,0.217282,0.787010,0,0.769843,0.012872,0.704043,0.003050,0.884905,0.422001,0.187121


In [ ]:
df.shape

(2050, 65)

In [ ]:
# Eliminar columna ID
df = df.drop(columns='ID')

# Explorar datos

In [ ]:
print('Number of duplicated', df.duplicated().sum())
print('Number of nan', df.isnull().sum().sum())

Number of duplicated 0
Number of nan 0


In [ ]:
# Dividir datos
X = df.drop('Bankruptcy', axis=1)
y = df['Bankruptcy']

In [ ]:
def display_correlation_matrix(corr_matrix):
    plt.figure(figsize=(20, 16))  # Adjust the size as needed
    sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', linewidths=.5, mask=np.triu(corr_matrix, k=1))
    plt.title('Correlation Matrix for Features')
    plt.show()

corr_matrix = X.corr()
display_correlation_matrix(corr_matrix)

In [ ]:
# Revisar variables con mayor y menor correlación
corr_matrix = X.corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

# Extract the upper triangle values as a series
upper_tri_series = corr_matrix.where(mask).stack()
# Exclude values of 1
upper_tri_series = upper_tri_series[upper_tri_series != 1]
print("Top 10 positive correlations:")
print(upper_tri_series.abs().sort_values(ascending=False).head(10))
print("\nTop 10 negative correlations:")
print(upper_tri_series.abs().sort_values(ascending=False).tail(10).sort_values(ascending=True))

In [ ]:
# Eliminar variables altamente correlacionadas
def remove_highly_correlated_features(corr_matrix, threshold=0.9):
    # Create a boolean mask for the upper triangle of the correlation matrix
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find index of feature columns with correlation greater than threshold
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]

    return to_drop

features_to_drop = remove_highly_correlated_features(corr_matrix, threshold=0.9)

print(f"Features to drop: {features_to_drop}")

X = X.drop(features_to_drop, axis=1)

print(f"New shape of X: {X.shape}")

In [ ]:
# Revisar nuevas correlaciones
corr_matrix = X.corr()
display_correlation_matrix(corr_matrix)

In [ ]:
y.value_counts().plot(kind='bar', title='Distribución bancarrota', color=['blue', 'red'], alpha=0.6, rot=0)
plt.show()
print('Compañías en bancarrota', y.value_counts()[1])
print('Compañías que no cayeron en bancarrota', y.value_counts()[0])
print('Ratio de compañías en bancarrota', y.value_counts()[1] / y.value_counts()[0])

En la gráfica anterior se puede observar que los datos están muy sesgados hacia las empresas que no están en quiebra.
Esto podría generar un mayor sesgo, prediciendo con mayor frecuencia la clase mayoritaria, lo cual puede generar una alta precisión, pero un bajo poder predictivo para la clase minoritaria.

Para corregir lo anterior, más adelante se buscará balancear un poco la clase minoritaria.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Train set:', len(X_train), len(X_train) / len(X_train) + len(X_test))
print('Test set:', len(X_test), len(X_test) / len(X_train) + len(X_test))

Ahora, para balancear las clases se usará SMOTE, que sirve para crear nuevas muestras de la clase minoritaria para que el modelo no se sesgue hacia la clase mayoritaria. Esto se hará únicamente para el set de entrenamiento.

In [ ]:
print('Antes', y_train.value_counts())
print('Ratio de compañías en bancarrota:', y_train.value_counts()[1] / y_train.value_counts()[0])

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print('SMOTE train set:', y_train_smote.value_counts())
print('Ratio de compañías en bancarrota:', y_train_smote.value_counts()[1] / y_train_smote.value_counts()[0])

In [ ]:
# Estandarizar X
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)

X_test_scaled = scaler.transform(X_test)

## Análisis de componentes proncipales

Se usará PCA para reducir la dimensionalidad en los datos

In [ ]:
pca = PCA(n_components=0.90) # retener el 90% de la varianza
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)


number_of_components = pca.n_components_
print('Número de componentes:', number_of_components)
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance_ratio = np.cumsum(explained_variance_ratio)

A continuación, se muestra la varianza acumulada explicada por los componentes

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, label='Varianza explicada')
plt.plot(range(1, len(cumulative_explained_variance_ratio) + 1), cumulative_explained_variance_ratio,
            label='Varianza explicada acumulada')
plt.axvline(number_of_components, color='red', linestyle='--')
plt.xlabel('Número de componentes')
plt.ylabel('Varianza')
plt.legend()
plt.grid()

# Red neuronal simple

In [ ]:
simple_parameters = {
    'hidden_layer_sizes': (10, 10, 10),
    'activation': 'tanh',
    'learning_rate_init': 0.01,
    'max_iter': 1000,
}

In [ ]:
simple_nn_classifier = MLPClassifier(**simple_parameters, random_state=42)

simple_nn_classifier.fit(X_train_pca, y_train_smote)

In [ ]:
simple_nn_test_score = simple_nn_classifier.score(X_test_pca, y_test)

print('Simple Neural Network Test Score:', simple_nn_test_score)

In [ ]:
from sklearn.metrics import roc_auc_score

# 1. Obtener probabilidades de predicción para la clase positiva (probabilidades, no etiquetas)
y_pred_proba = simple_nn_classifier.predict_proba(X_test_pca)[:, 1]

# 2. Calcular el AUC
auc_score = roc_auc_score(y_test, y_pred_proba)

# 3. Imprimir
print('AUC en el conjunto de prueba:', auc_score)

## Calibrar parámetros

In [ ]:
param_grid = {
    'hidden_layer_sizes': [(50, 50, 50), (100, 100, 100), (300, 300, 300)],
    'activation': ['tanh', 'relu', 'softmax', 'sigmoid'],
    'learning_rate_init': [0.01, 0.1],
    'max_iter': [500],
    'alpha': [0.0001, 0.001, 0.01], # L2 penalty (regularization term) parameter
    'solver': ['lbfgs', 'adam', 'sgd'], # The solver for weight optimization
}

In [ ]:
randomised_search_nn_classifier = RandomizedSearchCV(estimator=simple_nn_classifier, param_distributions=param_grid, n_iter=20, n_jobs=-1)

randomised_search_nn_classifier.fit(X_train_pca, y_train_smote)

best_params_nn_classifier = randomised_search_nn_classifier.best_params_

print('Best parameters neural network classifier:', best_params_nn_classifier)

In [ ]:
tuned_nn_classifier = MLPClassifier(**best_params_nn_classifier, random_state=42)

tuned_nn_classifier.fit(X_train_pca, y_train_smote)

tuned_nn_test_score = tuned_nn_classifier.score(X_test_pca, y_test)

print('Tuned Neural Network Test Score:', tuned_nn_test_score)

In [ ]:
# 1. Obtener probabilidades de predicción para la clase positiva (probabilidades, no etiquetas)
y_pred_2 = tuned_nn_classifier.predict_proba(X_test_pca)[:, 1]

# 2. Calcular el AUC
auc_score = roc_auc_score(y_test, y_pred_2)

# 3. Imprimir
print('AUC en el conjunto de prueba:', auc_score)

# Predicciones

In [ ]:
df_test=pd.read_csv("test_data.csv", index_col=0)

In [ ]:
# Eliminamos las mismas columnas que se eliminaron en el conjunto de entrenamiento
df_test = df_test.drop(columns=['No.credit.Interval', 'Working.Capital.Equity', 'Pre.tax.net.Interest.Rate',
                                'After.tax.Net.Profit.Growth.Rate', 'Inventory.and.accounts.receivable.Net.value',
                                'Continuous.interest.rate..after.tax.', 'Operating.Gross.Margin',
                                'Operating.Profit.Growth.Rate', 'Cash.Flow.to.Liability'])

In [ ]:
# Escalar los datos (usando el mismo scaler que se usó para X_train)
X_test_scaled = scaler.transform(df_test)

In [ ]:
# Aplicar la misma transformación PCA al conjunto de test
X_test_pca = pca.transform(X_test_scaled)

In [ ]:
y_test_pred = tuned_nn_classifier.predict(X_test_pca)

In [ ]:
# Crear el DataFrame con las predicciones
predicciones = pd.DataFrame({
    'ID': df_test.index,           # Recupera el índice como ID
    'Bankruptcy': y_test_pred      # Las clases predichas (0 o 1)
})

# Asegurar que el ID quede como índice si lo necesitás así
predicciones.set_index('ID', inplace=True)

# Mostrar las primeras filas
print(predicciones.head())

In [ ]:
predicciones.to_csv('predicciones.csv')